In [1]:
## O seguite código só serve para importar o SPARK
## Se der para mudar o Kernal para o SPARK, pode ser retirado

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
%config IPCompleter.greedy=True

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

## Imports

In [2]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

## <a id='2'> Ler Datasets </a>

In [3]:
rawUserArtistData = sc.textFile("profiledata_06-May-2005/user_artist_data.txt")
rawArtistData = sc.textFile("profiledata_06-May-2005/artist_data.txt")
rawArtistAlias = sc.textFile("profiledata_06-May-2005/artist_alias.txt")

## <a id='2'> Helper Functions </a>

As seguintes funções são usadas depois pelas Main Functions, as principais resposáveis pelo funcinamento do programa.
Por isso, mudámos a ordem em que as funções são apresentadas, metendo as Helper antes das outras.

## buildArtistById

In [20]:
#no livro é usado o flatMap, mas o resultado aqui é identico

def buildArtistByID(rawArtistData):
    artistDF = rawArtistData.map(lambda line: line.split("\t",1) if len(line.split("\t"))>1 else None).filter(lambda value: value is not None).toDF(["id","name"])
    artistDF = artistDF.withColumn("id", artistDF["id"].cast(IntegerType()))
    return artistDF

## buildArtistAlias

In [29]:
def buildArtistAlias(rawArtistAlias):
    artistAliasDF = rawArtistAlias.map(lambda line: line.split("\t",1) if line.split("\t")[0]!="" else None).filter(lambda value: value is not None).toDF(["artist","alias"])

    artistAliasDF = artistAliasDF.withColumn("artist", artistAliasDF["artist"].cast(IntegerType()))
    artistAliasDF = artistAliasDF.withColumn("alias", artistAliasDF["alias"].cast(IntegerType()))
    
    return artistAliasDF

## buildCounts

In [35]:
from pyspark.broadcast import Broadcast

## <a id='2'> Main Functions </a>

Principais funlções

## Preparation

In [5]:
def preparation(rawUserArtistData, rawArtistData, rawArtistAlias):
    userArtistDF = rawUserArtistData.map(lambda line: line.split(" ")).toDF(["user","artist"]).select("user", "artist")

    userArtistDF = userArtistDF.withColumn("user", userArtistDF["user"].cast(IntegerType()))
    userArtistDF = userArtistDF.withColumn("artist", userArtistDF["artist"].cast(IntegerType()))
    
    #userArtistDF.select(min("user"), max("user"), min("artist"), max("artist")).show()
    
    artistByID = buildArtistByID(rawArtistData)
    artistAlias = buildArtistAlias(rawArtistAlias)
    
    badID = artistAliasDF.head()[0]
    goodID = artistAliasDF.head()[1]
    
    #artistDF.filter(artistDF.id.isin(badID,goodID)).show()

In [30]:
artistAlias = buildArtistAlias(rawArtistAlias)

In [34]:
artistAlias.filter(artistAlias["alias"].isin(1000591)).show()

+-------+-------+
| artist|  alias|
+-------+-------+
|6789612|1000591|
|1262241|1000591|
|6791455|1000591|
|6694867|1000591|
|1295140|1000591|
|2127019|1000591|
|2153974|1000591|
|1232342|1000591|
|1270720|1000591|
|1238989|1000591|
|2051851|1000591|
|6794968|1000591|
|2055818|1000591|
|6676117|1000591|
|2008951|1000591|
|6679702|1000591|
|6682384|1000591|
|1330318|1000591|
|6646693|1000591|
|1290058|1000591|
+-------+-------+
only showing top 20 rows



## Model

## Evaluate

## Recommend